In [62]:
import xmltodict,easydict,math
parse = {}
with open("proto/msg_map/backend.xml",'r') as f:
    parse = easydict.EasyDict(xmltodict.parse(f.read()))
parse

{'protoCfg': {'@desc': '后端间通信',
  'cfg': [{'@target': 'System',
    '@up': 'C',
    '@down': 'S',
    '@resp_offset': '20000',
    '@type': 'normal',
    '@desc': '系统协议',
    'proto': [{'@id': '10001',
      '@name': 'HANDSHAKE',
      '@req': 'pb.S2CHandShake',
      '@resp': 'pb.C2SHandShake'},
     {'@id': '10002', '@name': 'ECHO', '@req': 'pb.S2CEcho'}]},
   {'@target': 'Server',
    '@up': 'C',
    '@down': 'S',
    '@type': 'sync',
    '@desc': '同步协议',
    'proto': [{'@id': '50001',
      '@name': 'SYNC_ERROR',
      '@req': 'pb.C2SSyncError'},
     {'@id': '50002',
      '@name': 'SYNC_GAME_SERVER',
      '@req': 'pb.C2SSyncGameServer'},
     {'@id': '50003',
      '@name': 'SYNC_ROOM_SERVER',
      '@req': 'pb.C2SSyncRoomServer'},
     {'@id': '50004',
      '@name': 'SYNC_WORLD_SERVER',
      '@req': 'pb.C2SSyncWorldServer'},
     {'@id': '50005',
      '@name': 'SYNC_FORWARD_MSG',
      '@req': 'pb.C2SForwardMsg'}]}]}}

In [79]:

## MsgID.lua
cli_id_title="""MsgID = {{
	{0}
}}
"""

## MsgPB.lua
cli_proto_title="""require "Assets/_LuaScripts/Game/Net/Proto/Importer"

local pb_scheme = {{
	{0}
}}

return pb_scheme
"""

## msg_name.go
srv_msg_title = """package msgid

var Name = map[uint16]string{{
	{0}
}}
"""

## *_msgproto.go
srv_proto_title = """package msgproto

import (
	"slotserver/common/msgid"
	"slotserver/common/pb"

	"google.golang.org/protobuf/reflect/protoreflect"
)

var {protoName} = map[uint16]protoreflect.MessageType{{
	{proto}
}}
"""

## *_msgid.go
srv_id_title = """package msgid

const (
	{0}
)
"""

fmt_name = "{a}2{b}_{name}"


fmt_cli_id = "{pname:{maxlen}s} = {pid},"
#fmt_cli_proto_name = '{pname}= "{pname}"'
fmt_cli_proto = '[MsgID.{pname}] = "{proto}",'

fmt_srv_id = "{pname:{maxlen}s} = {pid}"
fmt_srv_proto_name = '{{{pname}:}:{maxlen}s} "{pname}"'
fmt_srv_proto = "msgid.{pname}: (&{proto}).ProtoReflect().Type(),"


In [80]:
maxlen = 0
lcid = []
lcproto = []
lsid = []
lsprotoname = []
lsproto = []

for cfg in parse.protoCfg.cfg:
    up = cfg.get("@up",'')
    down = cfg.get("@down",'')
    resp_offset = int(cfg.get("@resp_offset",0))
    ctype = cfg.get("@type","normal")
    for proto in cfg.proto:
        proto_id = int(proto['@id'])
        name = proto['@name']
        req = proto.get('@req', None)
        resp = proto.get('@resp', None)
        if req:
            pname = fmt_name.format( a = up, b = down, name = name)
            maxlen = max(len(pname), maxlen)
        if resp:
            pname = fmt_name.format( a = down, b = up, name = name)
            maxlen = max(len(pname), maxlen)

for cfg in parse.protoCfg.cfg:
    up = cfg.get("@up",'')
    down = cfg.get("@down",'')
    resp_offset = int(cfg.get("@resp_offset",0))
    ctype = cfg.get("@type","normal")
    for proto in cfg.proto:
        proto_id = int(proto['@id'])
        name = proto['@name']
        req = proto.get('@req', None)
        resp = proto.get('@resp', None)
        if req:
            pname = fmt_name.format( a = up, b = down, name = name)
            cid = fmt_cli_id.format( pname = pname, pid = str(proto_id), maxlen = maxlen)
            cproto = fmt_cli_proto.format( pname = pname, proto = req, maxlen = maxlen)
            sid = fmt_srv_id.format( pname = pname, pid = str(proto_id), maxlen = maxlen)
            sproto = fmt_srv_proto.format( pname = pname, proto = req, maxlen = maxlen)
            sprotoname = fmt_srv_proto_name.format(pname = pname, maxlen = maxlen)
            lcid.append(cid)
            lcproto.append(cproto)
            lsid.append(sid)
            lsproto.append(sproto)
            lsprotoname.append(sprotoname)
        if resp:
            pname = fmt_name.format( a = down, b = up, name = name)
            pid = str( proto_id + resp_offset )
            cid = fmt_cli_id.format( pname = pname, pid = pid, maxlen = maxlen)
            cproto = fmt_cli_proto.format( pname = pname, proto = req, maxlen = maxlen)
            sid = fmt_srv_id.format( pname = pname, pid = pid, maxlen = maxlen)
            sproto = fmt_srv_proto.format( pname = pname, proto = resp, maxlen = maxlen)
            sprotoname = fmt_srv_proto_name.format(pname = pname, maxlen = maxlen)
            lcid.append(cid)
            lcproto.append(cproto)
            lsid.append(sid)
            lsproto.append(sproto)
            lsprotoname.append(sprotoname)




ValueError: Single '}' encountered in format string

In [78]:
with open('msgid.go','w') as f:
    f.write(srv_id_title.format(str.join('\n\t',lsid)))

with open('msgproto.go','w') as f:
    f.write(srv_proto_title.format(protoName="srv", proto=str.join('\n\t',lsproto)))

with open('MsgID.lua','w') as f:
    f.write(cli_id_title.format(str.join('\n\t',lcid)))

with open('MsgPB.lua','w') as f:
    f.write(cli_proto_title.format(str.join('\n\t',lcproto)))


SyntaxError: f-string expression part cannot include a backslash (523314557.py, line 2)